# **INSTALL LIBRARY**

In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 35.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 65.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 24.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=e821242dac31ee057c65c955880f97ad1f01605c9f6665ba79ecf29fd7e0c84d
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=5ac5482485d2abebae465ea99039f6403cf958b781c172df29432e0a2be07e36

# **IMPORT LIBRARY**

In [2]:
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-5daz6d8q
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-5daz6d8q
  Resolved https://github.com/huggingface/transformers.git to commit 66ded238cd04e29ba98485984dd647e7d37d1603
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7265129 sha256=03159e01ca2b73c2af28813c4701ac7118afdf98556385eb4efa6f348d28f37c
  Stored in directory: /tmp/pip-ephem-wheel-cache-4wobggg_/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.1
    Uninstalling transformers-4.30.1:
      Successfully uninstalled transformers-4.30.1
  Cloning https://github.com

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
from io import BytesIO
import pandas as pd
import numpy as np
import requests
import importlib.util
import torch
from ast import literal_eval
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
np.random.seed(0)
plt.style.use("ggplot")
import tensorflow as tf
from simpletransformers.ner import NERModel, NERArgs

print('Tensorflow version:', tf.__version__)
print('GPU detected:', tf.config.list_physical_devices('GPU'))

Tensorflow version: 2.12.0
GPU detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# **LOAD DATA**

In [39]:
# Efficiently load training, dev, and test data sets for train and optimal model evaluation

train_data = pd.read_csv('/kaggle/input/tung2121211212/data/Sequence_labeling_based_version/Syllable/train_BIO_syllable.csv')
dev_data = pd.read_csv('/kaggle/input/tung2121211212/data/Sequence_labeling_based_version/Syllable/dev_BIO_syllable.csv')
test_word_tag = pd.read_csv('/kaggle/input/tung2121211212/data/Sequence_labeling_based_version/Syllable/test_BIO_syllable.csv')

# Delete redundant column
train_data.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
dev_data.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
test_word_tag.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

# **BASIC PROCESS DATA BEFORE TRAINING**

In [40]:
# Fill Null vale

train_data.replace(np.nan, "NULL", inplace=True)
dev_data.replace(np.nan, "NULL", inplace=True)
test_word_tag.replace(np.nan, "NULL", inplace=True)

In [41]:
# Get number of tag in training dataset
tags = list(set(train_data["Tag"].values))
num_tags = len(tags)

In [42]:
train_data

,index,Word,Tag,sentence_id
0,0,Dừa,O,0
1,1,lắm,O,0
2,2,:)),O,0
3,4,Bấp,O,1
4,5,bênh,O,1
...,...,...,...,...
112749,121592,j,O,8843
112750,121593,thì,O,8843
112751,121594,kệ,O,8843
112752,121595,họ,O,8843


In [43]:
# Convert training set and dev set to Simpletransformer format

train_data = pd.DataFrame({'sentence_id': train_data['sentence_id'],'words': train_data['Word'], 'labels': train_data['Tag']})
dev_data = pd.DataFrame({'sentence_id': dev_data['sentence_id'],'words': dev_data['Word'], 'labels': dev_data['Tag']})

# **TRAINING MODEL**

In [44]:
%env JOBLIB_TEMP_FOLDER=/tmp

env: JOBLIB_TEMP_FOLDER=/tmp


In [45]:
num_examples = train_data.shape[0]  # replace with the size of your dataset
batch_size = 8  # replace with your batch size
N = 8  # replace with the number of epochs between each save

steps_per_epoch = num_examples / batch_size
save_steps = N * steps_per_epoch

args = NERArgs()
args.num_train_epochs = 16
args.learning_rate = 1.5e-5

args.save_steps = save_steps*2
args.save_model_every_epoch	 = False
args.save_eval_checkpoints = False
args.overwrite_output_dir = True

args.train_batch_size = 8
args.eval_batch_size = 8
args.use_cached_eval_features = False
args.use_multiprocessing = False
args.reprocess_input_data = True

args.use_early_stopping = True
args.early_stopping_delta = 0.01
args.early_stopping_metric = "mcc"
args.early_stopping_metric_minimize = False
args.early_stopping_patience = 5

In [46]:
model = NERModel("xlmroberta", "xlm-roberta-large", labels=tags, args=args)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [47]:
model.train_model(train_data, validation_data=dev_data, acc=accuracy_score)


/opt/conda/lib/python3.10/site-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/8844 [00:00<?, ?it/s]

Epoch:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 0 of 16:   0%|          | 0/1106 [00:00<?, ?it/s]

Running Epoch 1 of 16:   0%|          | 0/1106 [00:00<?, ?it/s]

...

Running Epoch 12 of 16:   0%|          | 0/1106 [00:00<?, ?it/s]

Running Epoch 13 of 16:   0%|          | 0/1106 [00:00<?, ?it/s]

Running Epoch 14 of 16:   0%|          | 0/1106 [00:00<?, ?it/s]

Running Epoch 15 of 16:   0%|          | 0/1106 [00:00<?, ?it/s]

(17696, 0.14760328841021295)

# **EVALUATE AND PREDICT**

In [48]:
test_word_tag = pd.DataFrame({'sentence_id': test_word_tag['sentence_id'],'words': test_word_tag['Word'], 'labels': test_word_tag['Tag']})

In [49]:
test = pd.read_csv('/kaggle/input/tung2121211212/data/Test_data/test.csv')

In [50]:
def evaluate(model, test, test_word_tag):

    def IBO_pred(tokens):
        ibo_list = []
        value_list = []
        pred = model.predict(tokens)
        for i in range(len(pred[0])):
            ibo_pred = pred[0][i][0]
            value_pred = list(ibo_pred.keys())
            ibo_pred = list(ibo_pred.values())
            ibo_list.append(ibo_pred[0])
            value_list.append(value_pred[0])
        return value_list, ibo_list

    def IBO_pred_test(test):
        list_ibo = []
        list_value = []
        df_pred = pd.DataFrame()
        for i in range(len(test)):
            text = test['content'][i]
            tokens = text.split()
            ibo_, t = IBO_pred(tokens)
            list_ibo.append(ibo_)
            list_value.append(t)
        df_pred['Value_pred'] = list_value
        df_pred['IBO_pred'] = list_ibo
        return df_pred

    def tokenize(text, pos):
        tokens = text.split()
        alignment = []
        start = 0
        for t in tokens:
            res = text.find(t, start)
            alignment.append(pos[res:res + len(t)])
            start = res + len(t)
        assert len(tokens) == len(alignment)
        return tokens, alignment

    def y_pred(data, df_predict):
        index_pred = []
        for i in range(len(df_predict)):
            value_predict_i = df_predict['Value_pred'][i]

            text = data['content'][i]
            pos = [i for i in range(len(text))]

            tokens, alignment = tokenize(text, pos)
            df_point = pd.DataFrame()

            df_point['spans'] = [0 for i in range(len(text))]

            for i, token in enumerate(value_predict_i):

                if token == 'B-T' or token == 'I-T':
                    for ali in alignment[i]:
                        df_point['spans'][ali] = 1
            index_pred.append(list(df_point['spans']))
        return index_pred

    def y_true(data):
        index_true = []

        for i in range(len(data)):
            text = data['content'][i]
            pos = [i for i in range(len(text))]
            df_point = pd.DataFrame()
            df_point['spans'] = pos
            df_point['spans'] = 0

            if not data['index_spans'][i]:
                index_true.append(list(df_point['spans']))
            else:
                for j in data['index_spans'][i]:
                    df_point['spans'][j] = 1
                index_true.append(list(df_point['spans']))
        return index_true

    # Main evaluate function logic starts here

    result, model_outputs, wrong_predictions = model.eval_model(test_word_tag)

    test['index_spans'] = test['index_spans'].apply(literal_eval)

    df_ibo = IBO_pred_test(test)

    true = y_true(test)
    pred = y_pred(test, df_ibo)

    # Dataframe for saving evaluation metrics
    scores_f1_macro = []
    scores_f1_micro = []
    scores_precision_macro = []
    scores_precision_micro = []
    scores_recall_macro = []
    scores_recall_micro = []

    for i in range(len(true)):
        score_macro = precision_recall_fscore_support(true[i], pred[i], average='macro', zero_division=0)
        score_micro = precision_recall_fscore_support(true[i], pred[i], average='micro', zero_division=0)

        scores_f1_macro.append(score_macro[2])
        scores_f1_micro.append(score_micro[2])
        scores_precision_macro.append(score_macro[0])
        scores_precision_micro.append(score_micro[0])
        scores_recall_macro.append(score_macro[1])
        scores_recall_micro.append(score_micro[1])

    scores = pd.DataFrame()
    scores['eval_loss'] = [list(result.values())[0]]
    scores['F1_ner'] = [list(result.values())[1]]
    scores['F1-micro'] = [np.mean(scores_f1_micro)]
    scores['F1-macro'] = [np.mean(scores_f1_macro)]
    scores['Precision-macro'] = [np.mean(scores_precision_macro)]
    scores['Precision-micro'] = [np.mean(scores_precision_micro)]
    scores['Recall-macro'] = [np.mean(scores_recall_macro)]
    scores['Recall-micro'] = [np.mean(scores_recall_micro)]

    return scores


In [ ]:
# Load the script as a module
spec = importlib.util.spec_from_file_location('evaluate', '/kaggle/input/hello-real/evaluate.py')
predict = importlib.util.module_from_spec(spec)
spec.loader.exec_module(evaluate)

In [51]:
evaluate(model, test, test_word_tag)

/opt/conda/lib/python3.10/site-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/139 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/3 [00:00<?, ?it/s]

,eval_loss,F1_ner,F1-micro,F1-macro,Precision-macro,Precision-micro,Recall-macro,Recall-micro
0,0.668721,0.537867,0.873774,0.755112,0.789337,0.873774,0.765059,0.873774


# **ERROR DATAFRAME FOR EVALUATE RESULT**

In [43]:
def word_true(df):
    list_toxic_true = []
    for idx in range(len(test)):
        toxic_true = []
        m = df.loc[df['sentence_id'] == idx]
        m.reset_index(inplace=True)
        k = list(m['words'])
        for i in range(len(k)):
            if(i == (len(k) - 1)):
                if(m['labels'][i] == 'B-T'):
                    toxic_true.append(m['words'][i])
                elif(m['labels'][i] == 'O'):
                    continue
            else:
                if(m['labels'][i] == 'B-T' and m['labels'][i+1] == 'O'):
                    toxic_true.append(m['words'][i])
                elif(m['labels'][i] == 'B-T' and m['labels'][i+1] == 'B-T'):
                    toxic_true.append(m['words'][i])
                elif(m['labels'][i] == 'B-T' and m['labels'][i+1] == 'I-T'):
                    j = i + 1
                    list_word = m['words'][i]
                    while(m['labels'][j] == 'I-T'):
                        list_word = list_word + " " + m['words'][j]
                        if(j == (len(k) -1 )):
                            break;
                        else:
                            j = j + 1
                    toxic_true.append(list_word)
        list_toxic_true.append(toxic_true) 
    return list_toxic_true

def word_pred(df):
    pre_value =[]
    for i in range(len(df)):
        temp = []
        for j in range(len(df['Value_pred'][i])):
            if df['Value_pred'][i][j] == 'B-T' or df['Value_pred'][i][j] == 'I-T':
                temp.append(df['IBO_pred'][i][j])
        pre_value.append(temp)
    return pre_value


# Error dataframe
error = pd.DataFrame()
error['Content'] = test['content']
error['True'] = word_true(test_word_tag)
error['Pred'] = word_pred(df_ibo)

NameError: name 'df_ibo' is not defined

In [ ]:
error

In [28]:
# Load the script as a module
spec = importlib.util.spec_from_file_location('predict', '/kaggle/input/hello-real/predict.py')
predict = importlib.util.module_from_spec(spec)
spec.loader.exec_module(predict)

In [29]:
text = "tung dep trai deo gi"
t, ibo_, temp = predict.predict(model, text)
print(ibo_, t)
print("predict: ", temp)

  0%|          | 0/5 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

['tung', 'dep', 'trai', 'deo', 'gi'] ['O', 'O', 'O', 'B-T', 'O']
predict:  ['deo']


# **SAVE MODEL**

In [34]:
def save_checkpoint(epoch, model, model_type):
    """
    Save model checkpoint.
    :param epoch: epoch number
    :param model: model
    :param optimizer: optimizer
    """
    state = {'epoch': epoch,
             'model': model}
    filename = 'checkpoint_phobert' + model_type + '.pth.tar'
    torch.save(state, filename)

In [35]:
print("Saving checkpoint epoch:", 20)
save_checkpoint(20, model, "base")

Saving checkpoint epoch: 20


In [37]:
checkpoint = torch.load('/kaggle/working/checkpoint_phobertbase.pth.tar')
start_epoch = checkpoint['epoch'] + 1
print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
model_xlm_base = checkpoint['model']


Loaded checkpoint from epoch 21.



In [38]:
import os
os.chdir(r'/kaggle/working')

!tar -czf checkpoint.pth.tar 

from IPython.display import FileLink

FileLink(r'checkpoint_phobertbase.pth.tar')

tar: Cowardly refusing to create an empty archive
Try 'tar --help' or 'tar --usage' for more information.


/kaggle/working/checkpoint_phobertbase.pth.tar